# 9 머신러닝 기초(비지도 학습)

In [ ]:
# 데이터 가공·처리·분석 라이브러리
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 머신러닝 라이브러리
import sklearn

# 소수점 3자리까지 표시
%precision 3

In [ ]:
# k-means를 사용하기 위해 임포트
from sklearn.cluster import KMeans

# 데이터를 생성하기 위해 임포트
from sklearn.datasets import make_blobs

# 샘플 데이터 생성
# 주의: make_blobs은 두 개의 값을 반환하므로
# 한쪽은 사용하지 않는 _로 입력 받음
X, _ = make_blobs(random_state=10)

# 그래프 출력
# color 옵션으로 색 지정 가능
plt.scatter(X[:,0],X[:,1],color='black')

In [ ]:
# KMeans 클래스 초기화
kmeans = KMeans(init='random',n_clusters=3)

# 군집 중심점 계산
kmeans.fit(X)

# 군집번호 예측
y_pred = kmeans.predict(X)

In [ ]:
# concat로 데이터를 가로 방향으로 결합(axis=1 지정)
merge_data = pd.concat([pd.DataFrame(X[:,0]), pd.DataFrame(X[:,1]), pd.DataFrame(y_pred)], axis=1)

# 위의 데이터에 X축을 eature1, Y축을 eature2, 군집번호를 cluster로 컬럼 이름 설정
merge_data.columns = ['feature1','feature2','cluster']

# 군집 결과를 시각화
ax = None
colors = ['blue', 'red', 'green']
for i, data in merge_data.groupby('cluster'):
    ax = data.plot.scatter(x='feature1', y='feature2', color=colors[i],
                                           label=f'cluster{i}', ax=ax)

In [ ]:
# 웹 사이트에서 데이터를 다운로드 한 후 zip 파일 압축을 풀기 위한 라이브러리 임포트
import requests, zipfile
import io

# 데이터가 있는 url 지정
zip_file_url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip'

# 데이터 수집하고 압축을 품
r = requests.get(zip_file_url, stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
# 데이터 읽어 들이기
bank= pd.read_csv('bank-full.csv', sep=';')

# 처음 다섯 행 데이터 출력
bank.head()

In [ ]:
print('데이터 형식(x,y):{}'.format(bank.shape))
print('결측 값 수:{}'.format(bank.isnull().sum().sum()))

In [ ]:
from sklearn.preprocessing import StandardScaler

# 데이터 컬럼 선택
bank_sub = bank[['age','balance','campaign','previous']]

# 표준화
sc = StandardScaler()
sc.fit(bank_sub)
bank_sub_std = sc.transform(bank_sub)
bank_sub.info()

In [ ]:
# KMeans 클래스 초기화
kmeans = KMeans(init='random', n_clusters=6, random_state=0)

# 군집 중심 계산
kmeans.fit(bank_sub_std)

# 군집 번호를 판다스 Series 객체로 변환
labels = pd.Series(kmeans.labels_, name='cluster_number')

# 군집 번호와 데이터 개수 출력
print(labels.value_counts(sort=False))

# 그래프 출력
ax = labels.value_counts(sort=False).plot(kind='bar')
ax.set_xlabel('cluster number')
ax.set_ylabel('count')

In [ ]:
# 엘보우법으로 추정. 군집 수를 1에서 10까지 증가시켜 거리의 종합 계산
dist_list =[]
for i in range(1,10):
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(X)
    dist_list.append(kmeans.inertia_)
    
# 그래프 출력
plt.plot(range(1,10), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')

In [ ]:
# 엘보우법을 이용한 추정. 군집 수를 1에서 20까지 증가시켜 거리의 종합 계산
dist_list =[]
for i in range(1,20):
    kmeans= KMeans(n_clusters=i, init='random', random_state=0)
    kmeans.fit(bank_sub_std)
    dist_list.append(kmeans.inertia_)
    
# 그래프 출력
plt.plot(range(1,20), dist_list,marker='+')
plt.xlabel('Number of clusters')
plt.ylabel('Distortion')

In [ ]:
# 금융기관 데이터에 군집 번호 데이터 결합
bank_with_cluster = pd.concat([bank, labels], axis=1)

# 처음 다섯 행 출력
bank_with_cluster.head()

In [ ]:
# 구간 분할 간격 설정
bins = [15,20,25,30,35,40,45,50,55,60,65,100]

# 위의 기준으로 금융기관 데이터를 분할하고, qcut_age 변수에 각 데이터의 연령층 설정
qcut_age = pd.cut(bank_with_cluster.age, bins, right=False)

# 군집 번호와 연령층 결합
df = pd.concat([bank_with_cluster.cluster_number, qcut_age], axis=1)

# 군집 번호와 연령층 기준으로 집계하고 연령층을 컬럼으로 설정
cross_cluster_age = df.groupby(['cluster_number', 'age']).size().unstack().fillna(0)
cross_cluster_age

In [ ]:
# 구간 분할한 데이터의 수 세기
hist_age = pd.value_counts(qcut_age)
hist_age

In [ ]:
sns.heatmap(cross_cluster_age.apply(lambda x : x/x.sum(), axis=1), cmap='Blues')

In [ ]:
cross_cluster_job = bank_with_cluster.groupby(['cluster_number', 'job']).size().unstack().fillna(0)
cross_cluster_job

In [ ]:
sns.heatmap(cross_cluster_job.apply(lambda x : x/x.sum(), axis=1),cmap='Reds')

In [ ]:
from sklearn.preprocessing import StandardScaler

# RandomState 객체 생성
sample = np.random.RandomState(1)

#두 개의 난수 생성
X = np.dot(sample.rand(2, 2), sample.randn(2, 200)).T

# 표준화
sc = StandardScaler()
X_std = sc.fit_transform(X)

# 상관계수 계산 및 그래프 출력
print('상관계수{:.3f}:'.format(sp.stats.pearsonr(X_std[:, 0], X_std[:, 1])[0]))
plt.scatter(X_std[:, 0], X_std[:, 1])

In [ ]:
# 임포트
from sklearn.decomposition import PCA

# 주성분 분석
pca = PCA(n_components=2)
pca.fit(X_std)

In [ ]:
print(pca.components_)

In [ ]:
print('각 주성분의 분산:{}'.format(pca.explained_variance_))

In [ ]:
print('각 주성분의 분산 비율:{}'.format(pca.explained_variance_ratio_))

In [ ]:
# 파라미터 설정
arrowprops=dict(arrowstyle='->',
                linewidth=2,
                shrinkA=0, shrinkB=0)

# 화살표를 그리기 위한 함수
def draw_vector(v0, v1):
    plt.gca().annotate('', v1, v0, arrowprops=arrowprops)

# 원본 데이터 그래프
plt.scatter(X_std[:, 0], X_std[:, 1], alpha=0.2)

# 주성분 분석의 두 개 축을 화살표로 나타냄
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v)

plt.axis('equal');

In [ ]:
# 유방암 데이터를 읽어 들이기 위해 임포트
from sklearn.datasets import load_breast_cancer

# 유방암 데이터 입력
cancer = load_breast_cancer()

# 데이터를 malignant(악성)과 benign(양성)으로 구분
# malignant(악성)은 cancer.target이 0
malignant = cancer.data[cancer.target==0]

# benign(양성)은 cancer.target이 1
benign = cancer.data[cancer.target==1]

# malignant(악성)는 파란색, benign(양성)은 오렌지 색을 칠해 히스토그램 생성
# 그래프는 각 설명변수(mean radius, mean texture 등)와
# 목표변수와의 관계를 나타내는 히스토그램
fig, axes = plt.subplots(6,5,figsize=(20,20))
ax = axes.ravel()
for i in range(30):
    _,bins = np.histogram(cancer.data[:,i], bins=50)
    ax[i].hist(malignant[:,i], bins, alpha=.5)
    ax[i].hist(benign[:,i], bins, alpha=.5)
    ax[i].set_title(cancer.feature_names[i])
    ax[i].set_yticks(())
    
# 레이블 설정
ax[0].set_ylabel('Count')
ax[0].legend(['malignant','benign'],loc='best')
fig.tight_layout()

In [ ]:
# 표준화
sc = StandardScaler()
X_std = sc.fit_transform(cancer.data)

# 주성분 분석
pca = PCA(n_components=2)
pca.fit(X_std)
X_pca = pca.transform(X_std)

# 출력
print('X_pca shape:{}'.format(X_pca.shape))
print('Explained variance ratio:{}'.format(pca.explained_variance_ratio_))

In [ ]:
# 컬럼에 레이블을 지정. 1번째가 제1 주성분, 2번째가 제2 주성분
X_pca = pd.DataFrame(X_pca, columns=['pc1','pc2'])

# 위의 데이터에 목표변수(cancer.target)를 가로 방향으로 결합
X_pca = pd.concat([X_pca, pd.DataFrame(cancer.target, columns=['target'])], axis=1)

# 악성과 양성을 분리
pca_malignant = X_pca[X_pca['target']==0]
pca_benign = X_pca[X_pca['target']==1]

In [ ]:
# 악성 데이터 그래프 출력
ax = pca_malignant.plot.scatter(x='pc1', y='pc2', color='red', label='malignant');

# 양성 데이터 그래프 출력
pca_benign.plot.scatter(x='pc1', y='pc2', color='blue', label='benign', ax=ax);

In [ ]:
trans = pd.read_excel('Online Retail.xlsx', sheet_name='Online Retail')
trans.head()

In [ ]:
# InoivceNo의 첫 번째 문자를 추출해 cancel_flg 생성
trans['cancel_flg'] = trans.InvoiceNo.map(lambda x:str(x)[0])

# cancel_flg의 그룹을 나누고 집계
trans.groupby('cancel_flg').size()

In [ ]:
trans = trans[(trans.cancel_flg == '5') & (trans.CustomerID.notnull())]

In [ ]:
# StockCode별로 데이터 개수를 집계하고
# 상위 5개를 출력
trans['StockCode'].value_counts().head(5)

In [ ]:
# 모든 InvoiceNo를 추출하고 trans_all에 저장
trans_all = set(trans.InvoiceNo) 

# 상품 85123A를 구입한 구매 데이터를 trans_a에 저장
trans_a = set(trans[trans['StockCode']=='85123A'].InvoiceNo)
print(len(trans_a))

# 상품 85099B를 구입한 구매 데이터를 trans_b에 저장
trans_b = set(trans[trans['StockCode']=='85099B'].InvoiceNo)
print(len(trans_b))

# 상품 85123A와 85099B를 동시에 구입한 구매 데이터를 trans_ab에 저장
trans_ab = trans_a&trans_b
print(len(trans_ab))

In [ ]:
# 두 상품을 포함하는 trans_ab의 수를 출력
print('두 상품을 모두 포함하는 장바구니 수:{}'.format(len(trans_ab)))
print('두 상품을 모두 포함하는 장바구니가 전체에서 차지하는 비율:{:.3f}'.format(len(trans_ab)/len(trans_all)))

In [ ]:
print('상품 85123A를 포함하는 장바구니 수:{}'.format(len(trans_a)))
print('상품 85123A를 포함하는 장바구니가 전체에서 차지하는 비율:{:.3f}'.format(len(trans_a)/len(trans_all)))

In [ ]:
print('신뢰도:{:.3f}'.format(len(trans_ab)/len(trans_a)))

In [ ]:
print('신뢰도:{:.3f}'.format(len(trans_ab)/len(trans_b)))

In [ ]:
# 전체 장바구니에서 차지하는 상품 B의 구매 비율 계산
support_b = len(trans_b) / len(trans_all)

# 상품 A를 구매하고 상품 B도 구매한 비율 계산
confidence = len(trans_ab) / len(trans_a)

# 리프트 값 계산
lift = confidence / support_b
print('lift:{:.3f}'.format(lift))